In [1]:
# avg last freeze date
import pandas as pd
import noaa_datasets

In [2]:
# pull down master data
stations = noaa_datasets.Stations()

Retrieving ghcnd-stations.txt
Retrieving ghcnd-states.txt
Retrieving ghcnd-countries.txt
Reading ghcnd-stations.txt
Reading ghcnd-countries.txt
Reading ghcnd-states.txt
Applying data prep operations to enrich dataset
Success!


In [3]:
inventory = noaa_datasets.Inventory(stations)

Retrieving ghcnd-inventory.txt
Reading ghcnd-inventory.txt
Merging with Stations dataset
Success!


In [4]:
# figure out best datasets to use for our requirement
# these are ones with the lastyear = 2019
# and at least 30 years for YearCount
# also, we're only looking at low temps, which is TMIN
df = inventory.df
df = df[ df['Element']=='TMIN' ]
df = df[ df['YearCount']>30 ]
df = df[ df['LastYear']==2019 ]

# whole world have 8,000+ results
# let's filter to just the united states, still 4,311...
df = df[ df['CountryCode']=='US' ]

# let's pick a random 100 of these
df_samples = df.sample(100)

,index,StationID,Element,FirstYear,LastYear,Latitude,Longitude,Elevation,State,StationName,GSN_Flag,HCN_CRN_Flag,WMO_ID,CountryCode,CountryName,StateName,YearCount
351965,351965,USC00141612,TMIN,1977,2019,38.9408,-95.3397,298.4,KS,CLINTON LAKE,NaN,NaN,NaN,US,United States,KANSAS,42
283250,283250,USC00010764,TMIN,1977,2019,33.3953,-87.0078,135.6,AL,BESSEMER 3 WSW,NaN,NaN,NaN,US,United States,ALABAMA,42
411247,411247,USC00251450,TMIN,1961,2019,40.6944,-99.7008,719.9,NE,CANADAY STEAM PLT,NaN,NaN,NaN,US,United States,NEBRASKA,58
565504,565504,USS0006M23S,TMIN,1983,2019,37.3800,-106.5500,3352.8,CO,Lily Pond,NaN,NaN,NaN,US,United States,COLORADO,36
497069,497069,USC00411063,TMIN,1935,2019,33.2064,-97.7761,234.4,TX,BRIDGEPORT,NaN,NaN,NaN,US,United States,TEXAS,84


In [22]:
# now we need to pull the daily file information for 
# each of these 100 records
# this will take some time, which is why we reduced
# to just 100 in the first place!

# let's try the process with just a single station
# then we can iterate and concat
def c_to_f(row):
    # (0°C × 9/5) + 32 = 32°F
    return ( ((((row['VALUE']/10) * 9) / 5) + 32))

full_df = pd.DataFrame()

for station in df_samples['StationID']:
    df_daily = noaa_datasets.DailyFile(station)
    df_daily.df['Low Temp (f)'] = df_daily.df.apply(c_to_f, axis=1)
    full_df = pd.concat([full_df, df_daily.df])

Retrieving USC00141612.dly
Transposing USC00141612 dataset
Applying data prep operations
Success!
Retrieving USC00010764.dly
Transposing USC00010764 dataset
Applying data prep operations
Success!
Retrieving USC00251450.dly
Transposing USC00251450 dataset
Applying data prep operations
Success!
Retrieving USS0006M23S.dly
Transposing USS0006M23S dataset
Applying data prep operations
Success!
Retrieving USC00411063.dly
Transposing USC00411063 dataset
Applying data prep operations
Success!
Retrieving USW00013781.dly
Transposing USW00013781 dataset
Applying data prep operations
Success!
Retrieving USC00451504.dly
Transposing USC00451504 dataset
Applying data prep operations
Success!
Retrieving USC00202250.dly
Transposing USC00202250 dataset
Applying data prep operations
Success!
Retrieving USC00086065.dly
Transposing USC00086065 dataset
Applying data prep operations
Success!
Retrieving USW00023160.dly
Transposing USW00023160 dataset
Applying data prep operations
Success!
Retrieving USC001166

MemoryError: 

In [ ]:
len(full_df)